In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from catboost import Pool

from data import get_data
from train import train_loop

In [2]:
cats = [
    'repair',
    'night',
    'month',
    'weekday'
]

drop_cols = [
    'road_id',
    'road_km',
    'datetime'
]

In [3]:
train = get_data('data/raw/train1.csv')

In [4]:
clf, score = train_loop(train.drop(drop_cols, axis=1), cats=cats, class_weights=[1, 2, 4])

Размер выборки для обучения:  (121219, 22)
Размер выборки для тестирования:  (30305, 22)
F1-score: 0.4102188252329935
     predicted  truth
0.0      30080  29337
1.0        208    805
2.0         17    163


In [5]:
test = get_data('data/raw/test1.csv', target=False)

In [6]:
test_pool = Pool(test.drop(drop_cols, axis=1), cat_features=cats)
test['target'] = clf.predict(test_pool).astype(int)

In [7]:
test['target'].value_counts()

0    7315429
1      25351
2       2374
Name: target, dtype: int64

In [15]:
from time import time
prediction = test[['datetime','road_id','road_km','target']][test['target'] != 0]
prediction.to_csv(f'data/predictions/sol_{int(time())}.csv', index=False)

In [16]:
test.shape

(7343154, 14)

In [17]:
test.head()

,datetime,road_id,road_km,target,lane_count,volume,occupancy,speed,repair,man_damaged_num,car_damaged_num,night,month,weekday
0,2020-01-01 01:00:00,9,38,0,3.0,87.0,1.0,114.671875,0,918,16857.0,1,1,2
1,2020-01-01 01:00:00,9,38,0,3.0,207.0,1.0,115.708435,0,918,16857.0,1,1,2
2,2020-01-01 01:00:00,9,38,0,3.0,43.0,20.0,46.453125,0,918,16857.0,1,1,2
3,2020-01-01 01:00:00,9,38,0,3.0,195.0,1.0,97.796875,0,918,16857.0,1,1,2
4,2020-01-01 01:00:00,9,38,0,3.0,89.0,1.0,108.213440,0,918,16857.0,1,1,2
